- Use the credit default dataset and build a decision tree model and a random forest model for predicting "default" outcome based on the others. 
- What is the Gini score at the root?
- Tune the decision tree model to find the combination of max depth and the criterion that maximize the accuracy score.
- As per decision tree model, which are most important features? 
- Tune the random forest model to find the max depth, n_estimators, min_samples_leaf, and criterion that maximize the recall score.


In [2]:
import pandas as pd
from sklearn import * 
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
df = pd.read_csv("/data/credit-default.csv")
df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,...,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job
0,< 0 DM,6,critical,radio/tv,1169,unknown,> 7 yrs,4,single male,none,...,real estate,67,none,own,2,1,1,yes,yes,skilled employee
1,1 - 200 DM,48,repaid,radio/tv,5951,< 100 DM,1 - 4 yrs,2,female,none,...,real estate,22,none,own,1,2,1,none,yes,skilled employee
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 yrs,2,single male,none,...,real estate,49,none,own,1,1,2,none,yes,unskilled resident
3,< 0 DM,42,repaid,furniture,7882,< 100 DM,4 - 7 yrs,2,single male,guarantor,...,building society savings,45,none,for free,1,1,2,none,yes,skilled employee
4,< 0 DM,24,delayed,car (new),4870,< 100 DM,1 - 4 yrs,3,single male,none,...,unknown/none,53,none,for free,2,2,2,none,yes,skilled employee


In [7]:
%%time 
target = "default"
label_encoder = preprocessing.LabelEncoder()

y = label_encoder.fit_transform(df[target])
X = df.drop(columns=target)

X = pd.get_dummies(X, drop_first=True)


X_train, X_test, y_train, y_test = model_selection.train_test_split(X.values * 1.0, y
                                        , test_size = 0.3, random_state = 1)

pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=2, include_bias=False)),
    #("scaler", preprocessing.StandardScaler()),
    ("est", ensemble.RandomForestClassifier(random_state = 1
                            ,  min_impurity_decrease = 0.001, min_samples_leaf = 25
                                        , max_depth = 3))
])

param_grid = {
    "est__max_depth": np.arange(2, 30, 5),
    "est__n_estimators": np.arange(5, 50, 5),
    "est__min_samples_leaf": np.arange(5, 10, 2),
    "est__criterion": ["gini", "entropy"],
    "est__max_features": np.linspace(0.5, 0.9, 5)
}

gsearch = model_selection.GridSearchCV(pipe, param_grid, cv = 5, scoring = "recall")
gsearch.fit(X, y)


CPU times: user 5min 7s, sys: 182 ms, total: 5min 8s
Wall time: 5min 8s


In [8]:
gsearch.best_estimator_

Pipeline(memory=None,
     steps=[('est', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=12, max_features=0.7, max_leaf_nodes=None,
            min_impurity_decrease=0.001, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=None,
            oob_score=False, random_state=241, verbose=0, warm_start=False))])

In [9]:
gsearch.best_params_

{'est__criterion': 'entropy',
 'est__max_depth': 12,
 'est__max_features': 0.7,
 'est__min_samples_leaf': 5,
 'est__n_estimators': 5}

In [10]:
gsearch.best_score_

0.4566666666666666